In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="5" color="red" >ch11_데이터프레임과 시리즈(Pandas)</font></b>
# 9절. 데이터 그룹화 및 집계

In [1]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Groupby
cf. pivot_table

In [5]:
# 1) 그룹화 방법1
#       df.groupby(by=[그룹핑할 열])[함수적용해서 출력할 열들].기초통계함수()
#                  [함수적용해서 출력할 열들] 생략시 그룹핑 할 열을 제외한 모든 열 적용
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
29,4.7,3.2,1.6,0.2,setosa
35,5.0,3.2,1.2,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
70,5.9,3.2,4.8,1.8,versicolor
94,5.6,2.7,4.2,1.3,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
109,7.2,3.6,6.1,2.5,virginica
143,6.8,3.2,5.9,2.3,virginica


In [6]:
iris.groupby(by='Species').mean()[['Petal.Length','Petal.Width']]
iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean()

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [7]:
iris_grouped_sum = iris.groupby('Species').median()
iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [8]:
iris.groupby('Species', as_index=False).sum()

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [9]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [10]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [11]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') |
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [12]:
#       df.groupby(by=[그룹핑할 열])[함수적용해서 출력할 열들].기초통계함수()'
# 2) 그룹화 방법2
#       df.pivot_table(index=[그룹핑할 열],
#                      values=[함수적용해서 출력할 열들],
#                      aggfunc=함수(기본값:mean))

In [13]:
iris.groupby('Species')[['Petal.Length','Petal.Width']].mean()
iris.pivot_table(index='Species',
                 values=['Petal.Length','Petal.Width'],
                 aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [14]:
iris.groupby('Species').count() # 결측치를 제외한 값의 갯수
iris.pivot_table(index='Species',
                #  values=['Petal.Length','Petal.Width'],
                 aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [15]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


다중열로 그룹화

In [16]:
# iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [1]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [18]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [19]:
# 각 열의 임의의 데이터에 결측치를 할당
iris.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0


In [20]:
import random
print(random.sample(range(149),3))
for col in range(iris.shape[1]-2): # iris.shape[1]-2
    iris.iloc[random.sample(range(150),col+1), col] = np.nan


[143, 32, 5]


In [21]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  149 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [22]:
iris.groupby(by=['Species','num'])[['Petal.Length','Petal.Width']].count() # as_index = False 하면(groupby에) 인덱스로 들어감

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           24
versicolor 2              24           24
           3              25           25
virginica  4              23           24
           5              25           24

In [23]:
iris.pivot_table(index = ['Species','num'],
                 values= ['Petal.Length','Petal.Width'],
                 aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           24
versicolor 2              24           24
           3              25           25
virginica  4              23           24
           5              25           24

In [24]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  49.000000   50.000000  50.000000
             mean    5.006122    5.936000   6.588000
             std     0.356141    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  50.000000   48.000000  50.000000
             mean    3.428000    2.783333   2.974000
             std     0.379064    0.299171   0.322497
             min     2.300000    2.200000   2.200000
             25%     3.200000    2.575000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
Petal.Length count  50.000000   49.000000  48.000000
             mean    1.462000    4.263265   5.556250
             std     0.173664    0.474207   0.562255
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.400000   5.550000
             75%     1.575000    4.600000   5.900000
             max     1.900000    5.100000   6.900000
Petal.Width  count  49.000000   49.000000  48.000000
             mean    0.248980    1.326531   2.029167
             std     0.104328    0.199766   0.279786
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [25]:
iris.drop('num',axis=1, inplace=True)
iris

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 <-> 롱포맷
- 판다스 공식 문서의 melt :
https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7page) : https://vita.had.co.nz/papers/tidy-data.pdf

In [ ]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                   package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt 를 이용한 언피벗팅(와이드 포맷 -> 롱포맷)

In [37]:
air_melted = airquality.melt(id_vars=['Month','Day'],
                var_name='변수명', # 기본값 : variable
                value_name='값') # 기본값 : value
air_melted[::25]
air_melted.sort_values(by=['Month','Day'], inplace=True)
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [ ]:
# melt 연습하기 (부동산 와이드포맷 데이터를 롱포맷으로)
%ls C:/ai_x/download/shareData/부동산_250213/

Parameter format not correct - "_x".


In [44]:
import pandas as pd
df = pd.read_csv(r'C:\ai_x\download\shareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv',
                 encoding='cp949')
df_melted = df.melt(id_vars=['지역'],
                    var_name='연도월',
                    value_name='평당분양가')
df_melted

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098
...,...,...,...
352,전북,2015년8월,6580
353,전남,2015년8월,6289
354,경북,2015년8월,7037
355,경남,2015년8월,7665


In [47]:
new_df = df.melt(id_vars='지역',
           var_name='연도월',
           value_name='평당분양가')
new_df.sample()

,지역,연도월,평당분양가
276,광주,2015년4월,7914


## 10.2 pivot_table을 이용한 피벗팅(롱포맷 -> 와이드포맷)

In [45]:
# air_melted 를 와이드 포맷으로
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0


In [49]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                       columns='변수명',
                       values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

In [50]:
airquality2.head()

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3


# 11절. 데이터프레임(시리즈)에 함수 적용시키기
- apply(func, axis) : 축요소별(요소별) 함수 적용 (데이터프레임이나 시리즈에서 함수적용)
- applymap(func) : 각 요소별 함수 적용(데이터 프레임에서만 가능)
- map(func, list) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)
## 11.1 apply

In [54]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [79]:
def get_year(datestr):
    return int(datestr.split('-')[0])

In [80]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [81]:
get_year('1999-05-01')

1999

In [82]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [83]:
get_month('1999-05-01')

5

In [84]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [88]:
X = iris.iloc[:,:-2]
# X 의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.848993
Sepal.Width     3.065541
Petal.Length    3.732653
Petal.Width     1.195890
dtype: float64

In [89]:
X.iloc[0] -avg

Sepal.Length   -0.748993
Sepal.Width     0.434459
Petal.Length   -2.332653
Petal.Width    -0.995890
dtype: float64

In [90]:
# X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.75,0.43,-2.33,-1.0
1,-0.95,-0.07,-2.33,-1.0
2,-1.15,0.13,-2.43,-1.0
3,-1.25,0.03,-2.23,-1.0
4,-0.85,0.53,-2.33,-1.0
...,...,...,...,...
145,0.85,-0.07,1.47,1.1
146,0.45,-0.57,1.27,0.7
147,0.65,-0.07,1.47,0.8
148,0.35,0.33,1.67,1.1


## 11.2 applymap

In [91]:
X.applymap(np.round)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원 

In [98]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [ ]:
# 연습하기(apply, map)
# new_df의 연도월 데이터를 연도와 월을 int로 분리하여 컬럼을 추가하세요

def get_year(datestr):
    return int(datestr.split('년')[0])


In [126]:
new_df['연도'] = new_df['연도월'].apply(lambda datestr : int(datestr.split('년')[0]))
new_df.head()

,지역,연도월,평당분양가,year,month,연도
0,서울,2013년12월,18189,2013,12,2013
1,부산,2013년12월,8111,2013,12,2013
2,대구,2013년12월,8080,2013,12,2013
3,인천,2013년12월,10204,2013,12,2013
4,광주,2013년12월,6098,2013,12,2013


In [127]:
new_df['월'] = new_df['연도월'].apply(lambda datestr : int(datestr.split('년')[1][:-1]))
new_df.head()

,지역,연도월,평당분양가,year,month,연도,월
0,서울,2013년12월,18189,2013,12,2013,12
1,부산,2013년12월,8111,2013,12,2013,12
2,대구,2013년12월,8080,2013,12,2013,12
3,인천,2013년12월,10204,2013,12,2013,12
4,광주,2013년12월,6098,2013,12,2013,12


In [119]:
get_year = lambda datestr: int(datestr.split('년')[0])

In [120]:
get_month = lambda datestr: int(datestr.split('년')[1][:-1])

In [122]:
get_year('1999년05월')

1999

In [ ]:
new_df.drop(['연도','월'], axis=1, inplace=True)
new_df.head()

KeyError: "['year', 'month'] not found in axis"

In [133]:
new_df

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098
...,...,...,...
352,전북,2015년8월,6580
353,전남,2015년8월,6289
354,경북,2015년8월,7037
355,경남,2015년8월,7665


In [134]:
# 2차원 데이터 프레임의 apply 이용하는 방법

In [136]:
new_df.head(2)

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111


In [151]:
def year_month_append(df):
    df = df.copy() # 깊은 복사
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [153]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [155]:
new_df = new_df.apply(year_month_append, axis=1)
new_df.head()

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


# 12절, 일괄변경하기(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 평균값, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [156]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                   columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [ ]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [159]:
#결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [160]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [162]:
# 결측치를 열별 평균값으로 대체
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [161]:
df.mean(axis=0)

A    3.0
B    3.0
C    NaN
D    2.5
dtype: float64

In [165]:
# 결측치 대체를 2번째 행까지만 대체
print(values)
df.fillna(value=values, limit=2)

{'A': 85, 'B': 80, 'C': 75, 'D': 90}


,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


In [ ]:
# (1단계) iris 각 열마다 결측치를 20개씩 인위적으로 할당한다
        # 결측치가 각 열마다 20개씩 확인
# (2-1단계) 결측치를 각 열의 중위수로 대체한다 : fillna 사용
# (2-2단계) 결측치를 종별 열별 중위수(groupby, pivot_table)로 대체한다 : fillna 사용X


In [55]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
iris = sns.load_dataset('iris')
rowlen, collen = iris.shape
rowlen, collen

(150, 5)

In [56]:
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당후 확인
print('임의의 20개 index 추출 :',random.sample(range(rowlen), 20))

임의의 20개 index 추출 : [146, 33, 122, 64, 25, 116, 53, 59, 128, 125, 120, 79, 101, 112, 41, 57, 110, 72, 0, 14]


In [57]:
# 결측치 할당
for col in range(collen-1):
    iris.iloc[random.sample(range(rowlen), 20), col] = np.nan

In [67]:
# 결측치 확인 - iris.isna(), iris.isnull() :결측치인지 여부
# iris.isna().sum()
iris.isnull().sum()

sepal_length    20
sepal_width     20
petal_length    20
petal_width     20
species          0
dtype: int64

In [66]:
# 결측치가 하나라도 있는 행 출력
iris[iris['sepal_length'].isna() | iris['sepal_width'].isna() |
     iris['petal_length'].isna() | iris['petal_width'].isna()]

,sepal_length,sepal_width,petal_length,petal_width,species
1,4.9,3.0,NaN,0.2,setosa
3,NaN,NaN,1.5,0.2,setosa
4,5.0,NaN,1.4,0.2,setosa
6,NaN,3.4,1.4,0.3,setosa
7,5.0,NaN,1.5,0.2,setosa
...,...,...,...,...,...
137,NaN,3.1,5.5,1.8,virginica
138,6.0,3.0,4.8,NaN,virginica
140,NaN,3.1,5.6,2.4,virginica
143,6.8,NaN,5.9,2.3,virginica


In [45]:
# (1단계) iris 각 열마다 결측치를 20개씩 인위적으로 할당한다
        # 결측치가 각 열마다 20개씩 확인
import random
for col in range(iris.shape[1]):
    iris.iloc[random.sample(range(150),20),col] = np.nan

In [ ]:
# 1)
for col in range()

In [46]:
iris.info()
iris.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  130 non-null    float64
 1   sepal_width   130 non-null    float64
 2   petal_length  130 non-null    float64
 3   petal_width   130 non-null    float64
 4   species       130 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


,sepal_length,sepal_width,petal_length,petal_width,species
44,5.1,3.8,NaN,0.4,setosa


In [ ]:
# (2-1단계) 결측치를 각 열의 중위수로 대체한다 : fillna 사용
# iris.iloc[:,:-1].median(axis=0)

values = iris.median(numeric_only=True)
iris.fillna(value=values)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,5.8,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,NaN
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,4.4,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.0,5.4,2.3,virginica


In [88]:
# (2-2단계) 결측치를 종별 열별 평균(groupby, pivot_table)로 대체한다 : fillna 사용X

iris_mean = iris.groupby('species').mean()

iris_mean

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.024444,3.441860,1.469048,0.240476
versicolor,5.924444,2.772727,4.245238,1.336364
virginica,6.590000,2.953488,5.521739,2.020455


In [ ]:
iris_mean # iris_mean

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.024444,3.441860,1.469048,0.240476
versicolor,5.924444,2.772727,4.245238,1.336364
virginica,6.590000,2.953488,5.521739,2.020455


In [100]:
# iris 한행을 매개변수로 받아 결측치가 있는 경우 종별 열별 중위수로 대체하는 함수
def iris_fillna(irisrow):
    irisrow = irisrow.copy()
    for col, value in irisrow.items():
        if isinstance(value, (int, float, np.number)) and np.isnan(value):
            # print(irisrow[col], '이 결측치')
            irisrow[col] = iris_mean.loc[irisrow.species, col]
    return irisrow

In [103]:
iris_fillna(iris.iloc[2])

sepal_length       4.7
sepal_width        3.2
petal_length       1.3
petal_width        0.2
species         setosa
Name: 2, dtype: object

In [105]:
# apply 적용
iris.apply(iris_fillna, axis=1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.100000,3.50000,1.400000,0.2,setosa
1,4.900000,3.00000,1.469048,0.2,setosa
2,4.700000,3.20000,1.300000,0.2,setosa
3,5.024444,3.44186,1.500000,0.2,setosa
4,5.000000,3.44186,1.400000,0.2,setosa
...,...,...,...,...,...
145,6.700000,3.00000,5.200000,2.3,virginica
146,6.300000,2.50000,5.000000,1.9,virginica
147,6.500000,3.00000,5.200000,2.0,virginica
148,6.200000,3.40000,5.400000,2.3,virginica


## 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용

In [107]:
s = pd.Series([0,1,2,3,np.nan])
s.replace(np.nan,0)
s.replace([0,1], 99) # 0이나 1을 99로 변경
s.replace([0,1], [100,101]) # 0은 100으로, 1은 101로 변경

0    100.0
1    101.0
2      2.0
3      3.0
4      NaN
dtype: float64

In [111]:
df = pd.DataFrame({'A' : ['bat', 'foo', 'bait'],
                   'B' : ['abc', 'bar', 'xyz']})
df.replace(r'ba.+', 'XX', regex=True)

,A,B
0,XX,abc
1,foo,XX
2,XX,xyz


## 12.3 mask(조건), where(조건)
- where(조건) : 조건이 True인 요소만 출력하고, 그 외 요소는 결측치로 출력
- mask(조건) : 조건이 False인 요소만 출력하고, 그 외 요소는 결측치로 출력

In [112]:
s = pd.Series(range(10,20))
s

0    10
1    11
2    12
3    13
4    14
5    15
6    16
7    17
8    18
9    19
dtype: int64

In [113]:
s.where(s>15)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64

## 12.4 dropna
- 결측치가 있는 데이터 누락(행, 열)

In [114]:
df = pd.DataFrame({'name':['홍', '김', '이', np.nan],
                   'age': [np.nan, 20, 30, np.nan],
                   'born':['서울', '부산', np.nan, np.nan]})

df

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN
3,NaN,NaN,NaN


In [115]:
df.dropna(how='all') # 모두 결측치인 행만 삭제(axis=0 기본값)

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN


In [116]:
df.dropna() # 결측치가 하나라도 있는 행 삭제

,name,age,born
1,김,20.0,부산


In [ ]:
df.dropna(axis=1) # 결측치가 하나라도 있는 열 삭제

""
0
1
2
3


In [118]:
df.dropna(subset=['name','age'])

,name,age,born
1,김,20.0,부산
2,이,30.0,NaN


# 13절. 시리즈
- 1차원 데이터

In [119]:
s = pd.Series([1000,7000,5000],
              index=['mango','apple','banana'])
s

mango     1000
apple     7000
banana    5000
dtype: int64

In [121]:
s['mango'], s.mango

(1000, 1000)

In [124]:
s['mango'] = 2800 # 수정
s['orange'] = 4000 # 추가
s

mango     2800
apple     7000
banana    5000
orange    4000
dtype: int64

In [ ]:
s['mango':'banana'] # 슬라이싱 [from:to] from부터 to까지(to포함)

mango     2800
apple     7000
banana    5000
dtype: int64

In [127]:
s[ ['mango','banana'] ] # 인덱스를 리스트로 넣어야함

mango     2800
banana    5000
dtype: int64

In [128]:
s.drop('banana')

mango     2800
apple     7000
orange    4000
dtype: int64

In [129]:
s.sort_index()

apple     7000
banana    5000
mango     2800
orange    4000
dtype: int64

In [130]:
s.sort_values() # 오름차순정렬

mango     2800
orange    4000
banana    5000
apple     7000
dtype: int64

In [131]:
s.sort_values(ascending=False) # 내림차순정렬

apple     7000
banana    5000
orange    4000
mango     2800
dtype: int64

# 연습문제(실습형)

In [133]:
import pandas as pd
import numpy as np

In [134]:
import seaborn as sns
iris = sns.load_dataset('iris')
type(iris)

pandas.core.frame.DataFrame

In [135]:
# 1. iris 데이터에서 처음 다섯개 행만 출력하세요
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
# 2. iris 데이터를 데이터프레임 변수인 독립변수 X와 종속변수 y로 나누세요. 종속변수는 species 열입니다.
X = iris.drop('species', axis=1)
# y = iris['species'] # 시리즈로 나옴
y=iris.loc[:,'species':] # 데이터프레임

X,y
print(type(y))

<class 'pandas.core.frame.DataFrame'>


In [137]:
# 3. iris 데이터에서 처음 50개행을 빼내서 temp변수에 저장하세요
temp = iris.iloc[:50]

In [195]:
# 4. 3번에서 선택한 데이터프레임의 요약정보를 출력하세요. 모든 열에 대해 요약정보가 출력되어야 합니다.
temp.describe(include=['float64','object'])
temp.describe(include='all')

,sepal_length,sepal_width,petal_length,petal_width,species
count,50.00000,50.000000,50.000000,50.000000,50
unique,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.00600,3.428000,1.462000,0.246000,NaN
std,0.35249,0.379064,0.173664,0.105386,NaN
min,4.30000,2.300000,1.000000,0.100000,NaN
25%,4.80000,3.200000,1.400000,0.200000,NaN
50%,5.00000,3.400000,1.500000,0.200000,NaN
75%,5.20000,3.675000,1.575000,0.300000,NaN


In [139]:
# 5. versicolor종의 데이터만 iris_versicolor변수에 저장하세요
iris_versicolor = iris[iris['species'] == 'versicolor']
iris_versicolor

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor
55,5.7,2.8,4.5,1.3,versicolor
56,6.3,3.3,4.7,1.6,versicolor
57,4.9,2.4,3.3,1.0,versicolor
58,6.6,2.9,4.6,1.3,versicolor
59,5.2,2.7,3.9,1.4,versicolor


In [140]:
#  6. 2번의 X와 y변수를 합해서 iris_df데이터 프레임으로 만드세요
iris_df = pd.concat([X, y], axis=1)
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [141]:
# 7. iris 데이터의 각 열 평균값을 출력
Z = iris.iloc[:, :-1]
Z.mean()

sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
dtype: float64

In [142]:
# 8. iris 데이터의 각 열들 사이의 상관계수를 출력
Z.corr()

,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,-0.117570,0.871754,0.817941
sepal_width,-0.117570,1.000000,-0.428440,-0.366126
petal_length,0.871754,-0.428440,1.000000,0.962865
petal_width,0.817941,-0.366126,0.962865,1.000000


In [143]:
# 9. iris 데이터에서 각 요소들과 변수별 평균과의 차이를 출력
Z - Z.mean()

,sepal_length,sepal_width,petal_length,petal_width
0,-0.743333,0.442667,-2.358,-0.999333
1,-0.943333,-0.057333,-2.358,-0.999333
2,-1.143333,0.142667,-2.458,-0.999333
3,-1.243333,0.042667,-2.258,-0.999333
4,-0.843333,0.542667,-2.358,-0.999333
...,...,...,...,...
145,0.856667,-0.057333,1.442,1.100667
146,0.456667,-0.557333,1.242,0.700667
147,0.656667,-0.057333,1.442,0.800667
148,0.356667,0.342667,1.642,1.100667


In [144]:
# 10. iris 데이터의 종별 평균을 출력
iris.groupby('species').mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [165]:
# 11. iris 데이터에서 각 요소들과 종별 변수의 평균과의 차이를 출력. 각 종별로 3개씩 출력
species_mean = iris.groupby('species').mean()
species_mean

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [ ]:
X = iris.columns[:4]

def iris_minus(irisrow):
    irisrow = irisrow.copy()
    if irisrow['species'] == irisrow['species']:
        return irisrow[X] - species_mean.loc[irisrow['species'], X]
    else:
        return irisrow[X]
    


In [192]:
iris_ans = iris.apply(iris_minus, axis=1)
iris_ans['species'] = iris['species']
result = iris_ans.groupby('species').head(3)
result

,sepal_length,sepal_width,petal_length,petal_width,species
0,0.094,0.072,-0.062,-0.046,setosa
1,-0.106,-0.428,-0.062,-0.046,setosa
2,-0.306,-0.228,-0.162,-0.046,setosa
50,1.064,0.430,0.440,0.074,versicolor
51,0.464,0.430,0.240,0.174,versicolor
52,0.964,0.330,0.640,0.174,versicolor
100,-0.288,0.326,0.448,0.474,virginica
101,-0.788,-0.274,-0.452,-0.126,virginica
102,0.512,0.026,0.348,0.074,virginica
